In [25]:
%load_ext autoreload
%autoreload 2

import os
import time
import openai
from dotenv import load_dotenv

from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from bs4 import BeautifulSoup as bs

from method.ours.utils import create_driver, get_xpath, get_application_context
from method.ours.parse import parse_form
from method.ours.prompts import get_form_context
from method.ours.history import HistoryTable
from method.ours.constraints import Invalid
from method.ours.generation import (
    generate_constraints_for_input_groups,
    generate_value_for_input_group,
    generate_values_for_input_groups,
    fill_form_with_value_table,
    submit_form
)
from method.ours.feedback import (
    get_local_feedback,
    get_global_feedback
)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [2]:
load_dotenv()
openai.api_key = os.getenv("OPENAI_API_KEY")

# Global Variables
HEADLESS = False
TEXT_EMBEDDING_METHOD = 'ADA'
GRAPH_EMBEDDING_METHOD = 'NODE2VEC'

URL = 'https://www.webmd.com/drugs/2/index'
#URL = 'https://seatgeek.ca/'
# 'https://www.stubhub.ca/'
# 'https://app.invoicing.co/#/register'
# 'http://localhost:3000/default-channel/en-US/account/register/'
# 'http://localhost:9000/dashboard/discounts/sales/add'
# 'http://localhost:9000/dashboard/customers/add'
# 'http://localhost:8080/'
# 'https://www.budget.com/en/home'
# 'https://www.thetrainline.com/en-us'
# 'https://www.mbta.com/'
# 'https://resy.com/'
# 'https://www.yelp.com/'
# 'https://www.aa.com/homePage.do'
# 'https://www.jetblue.com/'
# 'https://www.united.com/en/us'
# 'https://www.aircanada.com/ca/en/aco/home.html'


def get_to_form(driver):
    try:
        driver.get(URL)
        
        '''
        try:
            time.sleep(2)
            WebDriverWait(driver, 10).until(
                EC.presence_of_element_located((
                    By.XPATH,
                    '//BODY/DIV[1]/MAIN[1]/DIV[1]/DIV[1]/DIV[1]/FORM[1]/DIV[2]/DIV[1]/INPUT[1]')
                )
            ).send_keys('admin@example.com')
            WebDriverWait(driver, 10).until(
                EC.presence_of_element_located((
                    By.XPATH,
                    '//BODY/DIV[1]/MAIN[1]/DIV[1]/DIV[1]/DIV[1]/FORM[1]/DIV[4]/DIV[1]/DIV[1]/INPUT[1]')
                )
            ).send_keys('admin')
            WebDriverWait(driver, 10).until(
                EC.presence_of_element_located((
                    By.XPATH,
                    '//BODY/DIV[1]/MAIN[1]/DIV[1]/DIV[1]/DIV[1]/FORM[1]/DIV[5]/BUTTON[1]')
                )
            ).click()
        except:
            pass
        
        WebDriverWait(driver, 10).until(
            EC.presence_of_element_located((
                By.XPATH,
                '//BODY/DIV[1]/MAIN[1]/DIV[1]/DIV[2]/MAIN[1]/FORM[1]/DIV[1]/DIV[2]/DIV[4]/DIV[2]/LABEL[1]/SPAN[1]/SPAN[1]/INPUT[1]')
            )
        ).click()
        '''
        
        '''
        # Pet Clinic - Add Owner
        time.sleep(2)
        WebDriverWait(driver, 10).until(
            EC.presence_of_element_located((
                By.XPATH,
                '//BODY/APP-ROOT[1]/DIV[1]/NAV[1]/DIV[1]/UL[1]/LI[2]')
            )
        ).click()
        WebDriverWait(driver, 10).until(
            EC.presence_of_element_located((
                By.XPATH,
                '//BODY/APP-ROOT[1]/DIV[1]/NAV[1]/DIV[1]/UL[1]/LI[2]/UL[1]/LI[2]')
            )
        ).click()
        '''
        
        '''
        # Budget - Reservation
        WebDriverWait(driver, 5).until(
            EC.presence_of_element_located((By.ID, 'PicLoc_value'))
        ).click()
        '''
        
        '''
        # MBTA
        time.sleep(2)
        WebDriverWait(driver, 5).until(
            EC.presence_of_element_located((By.XPATH, '//BODY/DIV[1]/DIV[2]/MAIN[1]/DIV[1]/DIV[1]/DIV[1]/DIV[2]/A[1]'))
        ).click()
        '''
        
        '''
        # AC - My Bookings
        time.sleep(2)
        WebDriverWait(driver, 5).until(
            EC.presence_of_element_located((By.ID, 'bkmg-tab-button-mngBook'))
        ).click()
        '''
        
        '''
        # AC - Multi-city
        time.sleep(2)
        WebDriverWait(driver, 5).until(
            EC.presence_of_element_located((By.ID, 'bkmgFlights_tripTypeSelector_M'))
        ).click()
        '''
        
    except:
        print('timeout')


def find_form():
    return WebDriverWait(driver, 10).until(
        EC.presence_of_element_located((
            By.XPATH,
            '//BODY/DIV[1]/DIV[1]/DIV[2]/DIV[3]/DIV[1]/DIV[2]/DIV[1]/DIV[1]'
        ))
    )


def find_button():
    return None

In [3]:
from transformers import AutoTokenizer, pipeline, logging
from auto_gptq import AutoGPTQForCausalLM, BaseQuantizeConfig

def load_model_llama(model_type, use_triton=False):
    if model_type == '70B':
        model_name_or_path = "TheBloke/Llama-2-70B-chat-GPTQ"
        model_basename = "gptq_model-4bit--1g"
    elif model_type == '13B':
        model_name_or_path = "TheBloke/Llama-2-13B-chat-GPTQ"
        model_basename = "gptq_model-4bit-128g"
    else:
        raise ValueError('Invalid model_type. Choose either "70B" or "13B".')

    tokenizer = AutoTokenizer.from_pretrained(model_name_or_path, use_fast=True, device_map="auto")

    model = AutoGPTQForCausalLM.from_quantized(model_name_or_path,
                                               #model_basename=model_basename,
                                               revision = "main",
                                               inject_fused_attention=False,
                                               use_safetensors=True,
                                               trust_remote_code=False,
                                               use_triton=use_triton,
                                               quantize_config=None)
    return tokenizer, model


# Load the model - load_model_llama2_70b
tokenizer, model = load_model_llama("70B")

skip module injection for FusedLlamaMLPForQuantizedModel not support integrate without triton yet.                                                                                   


In [4]:
driver = create_driver(HEADLESS)
get_to_form(driver)

timeout


In [5]:
form = find_form()

In [6]:
history_table = HistoryTable(
    url=URL,
    xpath=get_xpath(driver, form)
)

In [7]:
for element in form.find_elements(By.TAG_NAME, 'input'):
    try:
        element.clear()
    except:
        pass

# Processing

In [8]:
html = driver.find_element(By.TAG_NAME, 'body').get_attribute('outerHTML')
relation_graph, input_groups = parse_form(
    driver,
    form,
    TEXT_EMBEDDING_METHOD=TEXT_EMBEDDING_METHOD
)

100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:00<00:00,  5.10it/s]
/home/webtesting/miniconda3/envs/llama2/lib/python3.10/site-packages/numpy/core/fromnumeric.py:3504: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
/home/webtesting/miniconda3/envs/llama2/lib/python3.10/site-packages/numpy/core/_methods.py:129: RuntimeWarning: invalid value encountered in scalar divide
  ret = ret.dtype.type(ret / rcount)
/home/webtesting/miniconda3/envs/llama2/lib/python3.10/site-packages/numpy/core/_methods.py:206: RuntimeWarning: Degrees of freedom <= 0 for slice
  ret = _var(a, axis=axis, dtype=dtype, out=out, ddof=ddof,
/home/webtesting/miniconda3/envs/llama2/lib/python3.10/site-packages/numpy/core/_methods.py:163: RuntimeWarning: invalid value encountered in divide
  arrmean = um.true_divide(arrmean, div, out=arrmean,
/home/webtesting/miniconda

# Generation

In [9]:
try:
    app_context = get_application_context(driver)
except:
    app_context = ''

In [10]:
app_context

''

In [21]:
value_table = generate_constraints_for_input_groups(
   input_groups,
   app_context=app_context,
   model=model,
   tokenizer=tokenizer,
)

# value_table = generate_constraints_for_input_groups(
#     model, 
#     tokenizer,
#     input_groups,
#     ablation_inclusion=constraint_ablation
# )

generated constraints: <s> [INST] <<SYS>>
    Instructions:
Your task is to generate a set of constraints for web form fields. Your decisions must be made independently without seeking user assistance or additional information. If there are multiple ways to express constraints, use the least number of constraints to describe them. Only generate the constraints and refrain from explaining your answers. Only generate constraints for the input field in question, not those in the relevant information section. You must choose your constraints in the format of our modified version of the Jest library in JavaScript. The list of functions in this modified format are:
1. toBeEqual(value) # the input field value is exactly equal to the given value
2. toHaveLengthCondition(condition, value) # the length of the input field value matches the given condition
3. toBeTruthy() # the input field value is truthy and not empty (not false, 0, '', null, undefined, or NaN)
4. toHaveCompareCondition(condition

In [22]:
value_table.print()


        input field:
        input: <input placeholder="Enter medication name to search" type="text" value=""/>
        constraints:
        input field's length should be greater than 2 characters
input field's length should be less than 50 characters
input field should be alphabetic
input field should have uppercase characters
input field should have special characters
input field should be empty
input field should conform to the given condition: Medication name must be a valid drug name
input field should be equal to field(confirm-medication-name)
input field should be  2023-12-13
input field should be  2023-12-14
input field should be  2023-12-12
        


In [26]:
value_table = generate_values_for_input_groups(
    input_groups,
    value_table,
    app_context=app_context,
    model=model,
    tokenizer=tokenizer,
)

prompting to LLM:[INST] <<SYS>>
        Your task is to generate a value for a web form field based on the form field information and a set of constraints on the field.
Your decisions must be made independently without seeking user assistance or additional information.
For each user prompt, you need to generate one value that satisfies the constraints while keeping in mind the nature of the input from the available information.
Only generate value and refrain from explaining your answers.
Only generate value for the input field in question, and not the ones in the relevant information section.
When generating values, generate the ones that actually conform to the constraints, and refrain from altering real-world values to fit the constraint, e.g. do not change the value "New York" to "NewYork" to satisfy the constraint not to contain whitespace.
        <</SYS>>
    
        The following are all the labels in the form, which provide context for the functionality of the form:
    We ar

In [27]:
fill_form_with_value_table(driver, value_table, input_groups)

<input>Enter medication name to search</input> at y: (380, 440), x: (74, 776)
<s> [INST] <<SYS>>
        Your task is to generate a value for a web form field based on the form field information and a set of constraints on the field.
Your decisions must be made independently without seeking user assistance or additional information.
For each user prompt, you need to generate one value that satisfies the constraints while keeping in mind the nature of the input from the available information.
Only generate value and refrain from explaining your answers.
Only generate value for the input field in question, and not the ones in the relevant information section.
When generating values, generate the ones that actually conform to the constraints, and refrain from altering real-world values to fit the constraint, e.g. do not change the value "New York" to "NewYork" to satisfy the constraint not to contain whitespace.
        <</SYS>>
    
        The following are all the labels in the form, w

In [28]:
submit_form(driver, input_groups=input_groups, explicit_submit=find_button())

IndexError: list index out of range

In [ ]:
new_html = driver.find_element(By.TAG_NAME, 'body').get_attribute('outerHTML')
global_feedback = get_global_feedback(html, new_html, remove_form_children=False)

In [ ]:
history_table.add(
    value_table.get_values_dict(),
    'base',
    global_feedback,
    driver.current_url
)